<a href="https://colab.research.google.com/github/xloem/techsketball/blob/main/02-byt5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

starting_model_path = 'baffo32/pyc2py_alpha2'#'google/byt5-large' #'t5-base'#'t5-small'#'bigscience/T0pp'
train_tokenizer = False # baffo32/pyc2py_alpha already has a tokenizer that can output bytes

input_width = 512
# these are not t5 parameters?
train_batch_size = 8 # small for notebook
num_epochs = 1
training_seed = 0
learning_rate = 0.0001#3e-4
logging_steps = 128


#!pip3 install --upgrade jax[tpu] -f https://storage.googleapis.com/jax-releases/libtpu_releases.html
!pip3 install --upgrade jax
!pip3 install --upgrade jaxlib
!pip3 install deepspeed
!pip3 install transformers
!pip3 install flax
!pip3 install sentencepiece
!git clone https://github.com/xloem/techsketball && ln -s techsketball/* .
!apt-get install git-lfs
!git config --global user.email johndoe@example.com
!git config --global user.name 'John Doe'
!git config --global credential.helper store

!git-lfs install
!git clone https://huggingface.co/baffo32/pyc2py_alpha2 local_model

     |████████████████████████████████| 887 kB 5.3 MB/s 
  Created wheel for jax: filename=jax-0.2.28-py3-none-any.whl size=1028669 sha256=bb649cb4242576472fcdbe1bb98fcbd6464327801a2a9b51df1858007a7f13e3
  Stored in directory: /root/.cache/pip/wheels/e2/60/4c/0cf931b766116b73950d9b6fca5813a45789d45d412a8d7272
Successfully built jax
  Attempting uninstall: jax
    Found existing installation: jax 0.2.25
    Uninstalling jax-0.2.25:
      Successfully uninstalled jax-0.2.25
     |████████████████████████████████| 65.1 MB 167 kB/s 
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.1.71+cuda111
    Uninstalling jaxlib-0.1.71+cuda111:
      Successfully uninstalled jaxlib-0.1.71+cuda111


In [ ]:
import jax, jax.tools.colab_tpu, jaxlib
import os
if 'COLAB_TPU_ADDR' in os.environ:
  jax.tools.colab_tpu.setup_tpu()
  jaxlib.tpu_client_extension.TpuClient.runtime_type = 'stream_executor'
  backend = 'tpu'
else:
  backend = jax.default_backend()
import tensorflow as tf
per_device_batch_size = train_batch_size // jax.device_count() ############################
!nvidia-smi
jax.local_devices()

In [ ]:
from transformers import T5Tokenizer, FlaxT5ForConditionalGeneration 
import huggingface_hub
repo = huggingface_hub.Repository('local_model', clone_from=starting_model_path)
#try:
#    tokenizer = T5Tokenizer.from_pretrained('local_model')
#except:
#    tokenizer = T5Tokenizer.from_pretrained(starting_model_path)
try:
    model = FlaxT5ForConditionalGeneration.from_pretrained('local_model')
except:
    model = FlaxT5ForConditionalGeneration.from_pretrained(starting_model_path)

In [ ]:
model_call = jax.jit(model)

In [ ]:
# before data is generated we can import libraries to generate it from
import jax, jax.numpy as jnp
import numpy as np
import optax
import flax
import flax.training.common_utils
import flax.training.train_state
import tqdm
import time
import os
# ...
import transformers
import deepspeed
import scipy
import matplotlib
import sentencepiece
import dis

In [ ]:
import find_pycode
print('getting training data ...')
#tokenizerpfx = starting_model_path.replace('/','_') + '.'
#if not train_tokenizer:
#  import bytepreservingsentencepiece
#  tokenizer.sp_model = bytepreservingsentencepiece.SentencePieceProcessor(tokenizer.vocab_file, **tokenizer.sp_model_kwargs)
tokenizerpfx = ''
find_pycode.write_files('example.', tokenizerpfx, 512, None, 512, globals(), skip_if_exists = True, tokenize_binary = False, tokenize_source = False)
#if train_tokenizer:
#  tokenizer.save_pretrained('local_model')
# repo.push_to_hub(commit_message=f'commit-message', blocking=False)
train_data = find_pycode.read_files('example.', tokenizerpfx, 512, 512, tokenize_binary = True)

In [ ]:
#from tokenizers import ByteLevelBPETokenizer
#tokenizer = ByteLevelBPETokenizer()
#tokenizer.train_from_iterator((str for bytes, str in data_tuples), vocab_size=model.config.vocab_size, min_frequency=2) 

In [ ]:
jax.config.update('jax_log_compiles', True)
from jax.experimental.compilation_cache import compilation_cache
#compilation_cache.initialize_cache('local_model/cc', max_cache_size_bytes=32*2**30)
compilation_cache._cache = None

In [ ]:
#cmd_args = None
#model_engine, optimizer, _, _ = deepspeed.initialize(args=cmd_args,
#                                                     model=model,
#                                                     model_parameters=params)

In [ ]:
num_train_steps = len(train_data['input_ids']) // train_batch_size * num_epochs

rng = jax.random.PRNGKey(training_seed)

In [ ]:
def batch_from_indices(dataset : dict, indices):
  #print(dataset['input_ids'].shape, indices.shape)
  result = {
      key : jnp.stack(data[indices,:])
      for key, data in dataset.items()
  }
  # this change could be already put in the dataset passed by the function that produces it
  result['labels'] = result['decoder_input_ids']
  pad_token_id = 0 # tokenizer.pad_token_id
  result['decoder_input_ids'] = np.asarray(transformers.models.t5.modeling_flax_t5.shift_tokens_right(result['labels'], pad_token_id, model.config.decoder_start_token_id))
  return result

# are these t5 parameters?
linear_decay_lr_schedule_fn = optax.linear_schedule(init_value=learning_rate, end_value=0, transition_steps=num_train_steps)
adamw = optax.adamw(learning_rate=linear_decay_lr_schedule_fn, b1=0.9, b2=0.98, eps=1e-8, weight_decay=0.01)
state = flax.training.train_state.TrainState.create(apply_fn=model_call, params=model.params, tx=adamw)

# Replicate the train state on each device
p_state = flax.jax_utils.replicate(state)




In [ ]:
# separate forward function out during compilation
jitted_state_apply_fn = jax.jit(state.apply_fn)
jitted_p_state_apply_fn = jax.jit(p_state.apply_fn)

In [ ]:

# from run_t5_mlm_flax.py
dropout_rngs = jax.random.split(rng, jax.local_device_count())

# Define gradient update step fn
def train_step(state, batch, dropout_rng):#input_ids, attention_mask, labels, decoder_input_ids, decoder_attention_mask, dropout_rng):
    print('Compiling train_step first line')
    dropout_rng, new_dropout_rng = jax.random.split(dropout_rng)
    def loss_fn(params):
        print('Compiling loss_fn.train_step first line')
        labels = batch.pop('labels')

        #logits = state.apply_fn(
        #    input_ids = input_ids,
        #    attention_mask = attention_mask,
        #    decoder_input_ids = decoder_input_ids,
        #    decoder_attention_mask = decoder_attention_mask,
        #    params = params,
        #    dropout_rng = dropout_rng,
        #    train = True
        #).logits
        logits = state.apply_fn(**batch, params = params, dropout_rng = dropout_rng, train = True).logits
        #print(logits.shape)
        #assert len(logits[-1]) == tokenizer.vocab_size
        #logits = logits[0]

        # logits, labels, padding_mask=batch['decoder_attention_mask', label_smoothing_factor=0]
        # compute loss
        print('Compiling train_step.loss_fn loss')
        loss = optax.softmax_cross_entropy(logits, flax.training.common_utils.onehot(labels, logits.shape[-1]))
        print('Compiling train_step.loss_fn padding_mask')
        padding_mask = batch['decoder_attention_mask']
        loss = (loss * padding_mask).sum() / padding_mask.sum()

        #loss = state.apply_fn(**batch, params=params, dropout_rng=dropout_rng, train=True).loss

        return loss

    print('Compiling train_step value_and_grad call')
    grad_fn = jax.value_and_grad(loss_fn)
    print('Compiling train_step grad_fn')
    loss, grad = grad_fn(state.params)
    print('Compiling train_step pmean')
    grad = jax.lax.pmean(grad, "batch")
    print('Compiling train_step apply_gradients')
    new_state = state.apply_gradients(grads=grad)

    print('Compiling train_step metrics')
    metrics = jax.lax.pmean(
        {"loss": loss, "learning_rate": linear_decay_lr_schedule_fn(state.step)}, axis_name="batch"
    )

    return new_state, metrics, new_dropout_rng

# Create parallel version of the train step
p_train_step = jax.pmap(train_step, 'batch', donate_argnums=(0,), backend=backend)

print('Performing initial batchs to compile train step ...')
rng, input_rng = jax.random.split(rng)
num_train_samples = len(train_data['input_ids'])
train_samples_idx = jax.random.permutation(input_rng, jnp.arange(num_train_samples))
model_inputs = batch_from_indices(train_data, train_samples_idx[:train_batch_size])

# trying single device pass
state, train_metric, dropout_rngs = train_step(state, model_inputs, dropout_rng=dropout_rngs)

model_inputs = flax.training.common_utils.shard(model_inputs)
print('About to call p_train_step ...')
p_state, train_metric, dropout_rngs = p_train_step(p_state, model_inputs, dropout_rng=dropout_rngs)
train_metric = flax.jax_utils.unreplicate(train_metric)
print('Done.  First loss was', train_metric['loss'].mean())

In [ ]:


train_time = 0
epochs = tqdm.tqdm(range(num_epochs), desc="Epoch ... ", position=0)
for epoch in epochs:
    # ======================== Training ================================
    train_start = time.time()
    train_metrics = []

    # Create sampling rng
    rng, input_rng = jax.random.split(rng)

    # Generate an epoch by shuffling sampling indices from the train dataset
    num_train_samples = len(train_data['input_ids'])
    train_samples_idx = jax.random.permutation(input_rng, jnp.arange(num_train_samples))
    #train_batch_idx = generate_batch_splits(train_samples_idx, train_batch_size)

    # Gather the indexes for creating the batch and do a training step
    batches_tqdm = tqdm.tqdm(range(num_train_samples // train_batch_size), desc="Training...", position=1)
    for step, batch_idx in enumerate(batches_tqdm):
        #samples = [tokenized_datasets["train"][int(idx)] for idx in batch_idx]
        model_inputs = batch_from_indices(train_data, train_samples_idx[train_batch_size * batch_idx:train_batch_size * batch_idx + train_batch_size])
        #print('model_inputs are', {key:val.shape for key, val in model_inputs.items()})
        #model_inputs = data_collator(samples)

        # Model forward
        model_inputs = flax.training.common_utils.shard(model_inputs)
        p_state, train_metric, dropout_rngs = p_train_step(p_state, model_inputs, dropout_rng=dropout_rngs)
        train_metrics.append(train_metric)

        cur_step = epoch * (num_train_samples // train_batch_size) + step

        if cur_step % logging_steps == 0 and cur_step > 0 and jax.process_index() == 0:
            # Save metrics
            train_metric = flax.jax_utils.unreplicate(train_metric)
            train_time += time.time() - train_start
            #if has_tensorboard and jax.process_index() == 0:
            #    write_train_metric(summary_writer, train_metrics, train_time, cur_step)

            epochs.write(
                f"Loss: {train_metric['loss'].mean()}, Learning Rate: {train_metric['learning_rate'].mean()}"
            )

            train_metrics = []
        #if cur_step % (num_train_samples // 8) == 0:
            # save checkpoint
            if jax.process_index() == 0:
                params = jax.device_get(jax.tree_map(lambda x: x[0], state.params))
                model.save_pretrained('local_model', params=params)
                repo.push_to_hub(commit_message=f'step={step} loss={train_metric["loss"].mean()}', blocking=False)

In [ ]:
while True:
  print(repr(eval(input('>>> '), globals(), locals())))